In [55]:
from elasticsearch import Elasticsearch
from datetime import datetime
import pandas as pd
from sklearn import cross_validation
from sklearn.svm import LinearSVC
import numpy as np
es = Elasticsearch()

In [47]:
# Cargar datos

data = pd.read_excel('Informe 93 - HCDN (Respuestas) (1) (1).xlsx',sheetname='Respuestas de formulario 1')
gi = ~data.Tema.isnull()
gi = gi & (~data.Pregunta.isnull())
preguntas = data.Pregunta[gi]
temas = data.Tema[gi]
Npreg = len(preguntas)

# Definir funciones utiles
def tfidf(tf, df, N):
    "Calcula el peso tf-idf a partir de las frecuencias."
    return np.log(1+tf)*np.log(float(N)/df)

In [109]:
# Borrar indices en elasticsearch
es.indices.delete(index="mi_indice")

{u'acknowledged': True}

In [10]:
res = es.search(index="test-index", body={"query": {"match_all": {}}})

In [39]:
print(res['_source']['pregunta'])

Informe las distintas clases de estupefacientes que se elaboran en nuestro país, teniendo en cuenta la información (procesos judiciales) proporcionada por los distintos Juzgados Federales de todo el país y el relevamiento realizado por los Ministerios de Seguridad y Justicia y Derechos Humanos.
Discrimine dicha información:
-por provincia
-por forma de elaboración 
-por tipo de sustancia
-por cuantificación de volumen de cada sustancia
¿Cómo ha sido el desarrollo y evolución del tráfico de estupefacientes en los últimos 10 años en nuestro país? Informe por provincia y por tipo de droga.


In [111]:
acciones = []
for i,preg in enumerate(preguntas):
    acciones.append({"index" : { "_index" : "mi_indice", "_id" : str(i), "_type": "my_type"} })
    acciones.append({"pregunta" : preg})
res = es.bulk(acciones)

In [65]:
res = es.get(index="mi_indice", doc_type="my_type", id="3")
print(res['_source']['pregunta'])

Déficit Habitacional: ¿Existe algún relevamiento del gobierno nacional respecto del déficit habitacional existente en la República Argentina? 


In [36]:
temp = res['docs'][3]['term_vectors']['pregunta']['terms'].values()

map(lambda x: tfidf(x['term_freq'], x['doc_freq'],  628), temp)

[3.0242695102770285,
 4.4656285520316326,
 3.9851755381134311,
 2.3891451729052489,
 4.4656285520316326,
 3.5047225241952296,
 2.5438164963588266,
 4.4656285520316326,
 4.4656285520316326,
 5.8709048363072709]

In [49]:
vocabulario = [k for doc in res['docs'] for k, v in doc['term_vectors']['pregunta']['terms'].iteritems()]

In [110]:
# Crear un indice en elasticsearch

mapeos = {
  "mappings": {
    "my_type": {
      "properties": {
        "pregunta": { 
          "type": "string",
          "term_vector": "yes",
          "analyzer": "spanish",
          "similarity": "BM25",
          "fields": {
            "raw": { 
              "type":  "string",
              "index": "not_analyzed"
            }
           }
          }
        }
      }
    }
  }

res = es.indices.create(index="mi_indice",body=mapeos)

In [32]:
import numpy as np

In [25]:
from scipy import sparse

In [113]:
query = {"docs": [{"_id": str(num), "term_statistics": "true"} for num in range(len(preguntas))]}
res= es.mtermvectors(index="mi_indice", doc_type="my_type", body=query)
matrix_tfidf = sparse.csr_matrix((len(preguntas),len(vocabulario)))
for idx in range(len(preguntas)):
    terminos = res['docs'][idx]['term_vectors']['pregunta']['terms'].keys()
    valores =  res['docs'][idx]['term_vectors']['pregunta']['terms'].values()
    indices = map(vocabulario.index, terminos)
    tfidf_vec = map(lambda x: tfidf(x['term_freq'], x['doc_freq'],  628), valores)
    matrix_tfidf[idx,indices] = tfidf_vec

In [114]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
                matrix_tfidf, temas, test_size=0.5, random_state=0)
svm = LinearSVC()
svm.fit(X_train,y_train)
print "Score correctas: {:f}".format(svm.score(X_test,y_test))

Score correctas: 0.483384


In [40]:
len(res['docs'])

628

In [119]:
id = 8
res = es.get(index="mi_indice", doc_type="my_type", id=str(id))
print(res['_source']['pregunta'])
print('===========================================================')

query = {
        "more_like_this" : {
        "fields" : ["pregunta"],
        "like" : [
        {
            "_index" : "mi_indice",
            "_type" : "my_type",
            "_id" : str(id)
        }],
        "min_term_freq" : 1,
        "max_query_terms" : 25
    }}
res = es.search(index="mi_indice", doc_type="my_type", body={"from" : 0, "size" : 3, "query": query})

for hit in res['hits']['hits']:
    print(hit['_source']['pregunta'])
    print('//////////////////////////////// score:' + str(hit['_score']))

Para cualquier acción de emergencia y/o plan es necesario disponer de la dinámica de las estadísticas en forma urgente. ¿Cuándo estarán disponibles?
Respecto al tráfico y contrabando de estupefacientes, informe si nuestro país es de tránsito o de producción. Si fuera de producción: ¿Qué medidas adoptará el Poder Ejecutivo para evitarlo y cuáles serán las acciones necesarias para evitar el contrabando aéreo, terrestre, fluvial o marítimo que existe actualmente?
Asimismo, especifique los plazos de ejecución de dicho Plan de acción, si lo hubiere.
//////////////////////////////// score:6.2821302
Informe perspectivas y plan de acción del Poder Ejecutivo respecto del Plan Nacional de Erradicación de Violencia de Género (ley 26.485) en el presupuesto 2017?
//////////////////////////////// score:4.5231733
¿Cuándo y de qué forma van a articular los subsidios a los fletes al puerto para las economías regionales, también prometidas dentro del Plan Belgrano?
//////////////////////////////// score

In [95]:
np.flatnonzero(preguntas.str.find('PAMI')>0)

array([ 68,  69,  70,  71,  72,  73,  74,  75, 150, 347, 477, 478, 480])

In [99]:
hit

{u'_id': u'75',
 u'_index': u'mi_indice',
 u'_score': 0.28504398,
 u'_source': {u'pregunta': u'Sobre Instituto Nacional de Servicios Sociales para Jubilados y Pensionados (PAMI): \nEnuncie la cantidad de ambulancias que posee el PAMI, con indicaci\xf3n de modelo, a\xf1o de compra y estado de dichos transportes.-'},
 u'_type': u'my_type'}